# Run model module locally

In [ ]:
import math
import os

def convert_conv_layer_property_lists_to_string(property_list, prop_list_len):
    """Convert conv layer property list to string.

    Args:
        property_list: list, nested list of blocks of a conv layer property.
        prop_list_len: int, length of list to process.

    Returns:
        Doubly delimited string of conv layer property values.
    """
    return (";").join(
        [
            (",").join([str(val) for val in block])
            for block in property_list[0:prop_list_len]
        ]
    )
    

# Import os environment variables for file hyperparameters.
os.environ["TRAIN_FILE_PATTERN"] = "gs://machine-learning-1234-bucket/gan/pg_anogan_sim_enc/data/cifar10_car/train*.tfrecord"
os.environ["EVAL_FILE_PATTERN"] = "gs://machine-learning-1234-bucket/gan/pg_anogan_sim_enc/data/cifar10_car/test*.tfrecord"
os.environ["OUTPUT_DIR"] = "gs://machine-learning-1234-bucket/gan/pg_anogan_sim_enc/trained_model"

# Import os environment variables for train hyperparameters.
os.environ["TRAIN_BATCH_SIZE"] = str(16)
os.environ["TRAIN_STEPS"] = str(60)
os.environ["USE_TPU"] = "False"
os.environ["USE_ESTIMATOR_TRAIN_AND_EVALUATE"] = "False"
os.environ["SAVE_OPTIMIZER_METRICS_TO_CHECKPOINT"] = "True"
os.environ["SAVE_SUMMARY_STEPS"] = str(1)
os.environ["SAVE_CHECKPOINTS_STEPS"] = str(100)
os.environ["KEEP_CHECKPOINT_MAX"] = str(100)

# Import os environment variables for eval hyperparameters.
os.environ["EVAL_BATCH_SIZE"] = str(16)
os.environ["EVAL_STEPS"] = "None"
os.environ["START_DELAY_SECS"] = str(600)
os.environ["THROTTLE_SECS"] = str(600)
os.environ["EVAL_ON_TPU"] = "False"

# Import os environment variables for serving hyperparameters.
os.environ["EXPORTS_TO_KEEP"] = str(50)
os.environ["EXPORT_TO_TPU"] = "False"
os.environ["EXPORT_TO_CPU"] = "True"
os.environ["PREDICT_ALL_RESOLUTIONS"] = "True"
os.environ["PREDICT_G_Z"] = "True"
os.environ["ANOMALY_THRESHOLD"] = str(5.0)
os.environ["ANOM_CONVEX_COMBO_FACTOR"] = str(0.05)

# Import os environment variables for image hyperparameters.
os.environ["HEIGHT"] = str(32)
os.environ["WIDTH"] = str(32)
os.environ["DEPTH"] = str(3)

# Import os environment variables for shared hyperparameters.
os.environ["NUM_STEPS_UNTIL_GROWTH"] = str(10)
os.environ["USE_EQUALIZED_LEARNING_RATE"] = "True"

# Full lists for full 1024x1024 network growth.
full_conv_num_filters = [[512, 512], [512, 512], [512, 512], [512, 512], [256, 256], [128, 128], [64, 64], [32, 32], [16, 16]]
full_conv_kernel_sizes = [[4, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
full_conv_strides = [[1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1]]

# Set final image size as a multiple of 2, starting at 4.
image_size = 16
os.environ["IMAGE_SIZE"] = str(image_size)
prop_list_len = max(
    min(int(math.log(image_size, 2) - 1), len(full_conv_num_filters)), 1
)

# Get slices of lists.
conv_num_filters = convert_conv_layer_property_lists_to_string(
    full_conv_num_filters, prop_list_len
)
print("conv_num_filters = {}".format(conv_num_filters))
conv_kernel_sizes = convert_conv_layer_property_lists_to_string(
    full_conv_kernel_sizes, prop_list_len
)
print("conv_kernel_sizes = {}".format(conv_kernel_sizes))
conv_strides = convert_conv_layer_property_lists_to_string(
    full_conv_strides, prop_list_len
)
print("conv_strides = {}".format(conv_strides))

os.environ["CONV_NUM_FILTERS"] = conv_num_filters
os.environ["CONV_KERNEL_SIZES"] = conv_kernel_sizes
os.environ["CONV_STRIDES"] = conv_strides

# Import os environment variables for generator hyperparameters.
os.environ["LATENT_SIZE"] = str(512)
os.environ["NORMALIZE_LATENT"] = "True"
os.environ["USE_PIXEL_NORM"] = "True"
os.environ["PIXEL_NORM_EPSILON"] = str(1e-8)
os.environ["GENERATOR_PROJECTION_DIMS"] = "4,4,512"
os.environ["GENERATOR_LEAKY_RELU_ALPHA"] = str(0.2)
os.environ["GENERATOR_TO_RGB_ACTIVATION"] = "None"
os.environ["GENERATOR_L1_REGULARIZATION_SCALE"] = str(0.)
os.environ["GENERATOR_L2_REGULARIZATION_SCALE"] = str(0.)
os.environ["GENERATOR_OPTIMIZER"] = "Adam"
os.environ["GENERATOR_LEARNING_RATE"] = str(0.001)
os.environ["GENERATOR_ADAM_BETA1"] = str(0.)
os.environ["GENERATOR_ADAM_BETA2"] = str(0.99)
os.environ["GENERATOR_ADAM_EPSILON"] = str(1e-8)
os.environ["GENERATOR_CLIP_GRADIENTS"] = "None"
os.environ["GENERATOR_TRAIN_STEPS"] = str(1)

# Import os environment variables for discriminator hyperparameters.
os.environ["USE_MINIBATCH_STDDEV"] = "True"
os.environ["MINIBATCH_STDDEV_GROUP_SIZE"] = str(4)
os.environ["MINIBATCH_STDDEV_AVERAGING"] = "True"
os.environ["DISCRIMINATOR_LEAKY_RELU_ALPHA"] = str(0.2)
os.environ["DISCRIMINATOR_L1_REGULARIZATION_SCALE"] = str(0.)
os.environ["DISCRIMINATOR_L2_REGULARIZATION_SCALE"] = str(0.)
os.environ["DISCRIMINATOR_OPTIMIZER"] = "Adam"
os.environ["DISCRIMINATOR_LEARNING_RATE"] = str(0.001)
os.environ["DISCRIMINATOR_ADAM_BETA1"] = str(0.)
os.environ["DISCRIMINATOR_ADAM_BETA2"] = str(0.99)
os.environ["DISCRIMINATOR_ADAM_EPSILON"] = str(1e-8)
os.environ["DISCRIMINATOR_CLIP_GRADIENTS"] = "None"
os.environ["DISCRIMINATOR_GRADIENT_PENALTY_COEFFICIENT"] = str(10.0)
os.environ["EPSILON_DRIFT"] = str(0.001)
os.environ["DISCRIMINATOR_TRAIN_STEPS"] = str(1)

# Import os environment variables for encoder hyperparameters.
os.environ["ENCODER_LEAKY_RELU_ALPHA"] = str(0.2)
os.environ["ENCODER_L1_REGULARIZATION_SCALE"] = str(0.)
os.environ["ENCODER_L2_REGULARIZATION_SCALE"] = str(0.)
os.environ["ENCODER_OPTIMIZER"] = "Adam"
os.environ["ENCODER_LEARNING_RATE"] = str(0.001)
os.environ["ENCODER_ADAM_BETA1"] = str(0.)
os.environ["ENCODER_ADAM_BETA2"] = str(0.99)
os.environ["ENCODER_ADAM_EPSILON"] = str(1e-8)
os.environ["ENCODER_CLIP_GRADIENTS"] = "None"


## Train pg_AnoGAN_Sim_Enc model

In [ ]:
%%bash
gsutil -m rm -rf ${OUTPUT_DIR}
export PYTHONPATH=$PYTHONPATH:$PWD/pg_anogan_sim_enc_tpu_module
python3 -m trainer.task \
    --train_file_pattern=${TRAIN_FILE_PATTERN} \
    --eval_file_pattern=${EVAL_FILE_PATTERN} \
    --output_dir=${OUTPUT_DIR} \
    --job-dir=./tmp \
    \
    --train_batch_size=${TRAIN_BATCH_SIZE} \
    --train_steps=${TRAIN_STEPS} \
    --use_tpu=${USE_TPU} \
    --use_estimator_train_and_evaluate=${USE_ESTIMATOR_TRAIN_AND_EVALUATE} \
    --save_optimizer_metrics_to_checkpoint=${SAVE_OPTIMIZER_METRICS_TO_CHECKPOINT} \
    --save_summary_steps=${SAVE_SUMMARY_STEPS} \
    --save_checkpoints_steps=${SAVE_CHECKPOINTS_STEPS} \
    --keep_checkpoint_max=${KEEP_CHECKPOINT_MAX} \
    \
    --eval_batch_size=${EVAL_BATCH_SIZE} \
    --eval_steps=${EVAL_STEPS} \
    --start_delay_secs=${START_DELAY_SECS} \
    --throttle_secs=${THROTTLE_SECS} \
    --eval_on_tpu=${EVAL_ON_TPU} \
    \
    --exports_to_keep=${EXPORTS_TO_KEEP} \
    --export_to_tpu=${EXPORT_TO_TPU} \
    --export_to_cpu=${EXPORT_TO_CPU} \
    --predict_all_resolutions=${PREDICT_ALL_RESOLUTIONS} \
    --predict_g_z=${PREDICT_G_Z} \
    --anomaly_threshold=${ANOMALY_THRESHOLD} \
    --anom_convex_combo_factor=${ANOM_CONVEX_COMBO_FACTOR} \
    \
    --height=${HEIGHT} \
    --width=${WIDTH} \
    --depth=${DEPTH} \
    \
    --num_steps_until_growth=${NUM_STEPS_UNTIL_GROWTH} \
    --use_equalized_learning_rate=${USE_EQUALIZED_LEARNING_RATE} \
    --conv_num_filters=${CONV_NUM_FILTERS} \
    --conv_kernel_sizes=${CONV_KERNEL_SIZES} \
    --conv_strides=${CONV_STRIDES} \
    \
    --latent_size=${LATENT_SIZE} \
    --normalize_latent=${NORMALIZE_LATENT} \
    --use_pixel_norm=${USE_PIXEL_NORM} \
    --pixel_norm_epsilon=${PIXEL_NORM_EPSILON} \
    --generator_projection_dims=${GENERATOR_PROJECTION_DIMS} \
    --generator_leaky_relu_alpha=${GENERATOR_LEAKY_RELU_ALPHA} \
    --generator_to_rgb_activation=${GENERATOR_TO_RGB_ACTIVATION} \
    --generator_l1_regularization_scale=${GENERATOR_L1_REGULARIZATION_SCALE} \
    --generator_l2_regularization_scale=${GENERATOR_L2_REGULARIZATION_SCALE} \
    --generator_optimizer=${GENERATOR_OPTIMIZER} \
    --generator_learning_rate=${GENERATOR_LEARNING_RATE} \
    --generator_adam_beta1=${GENERATOR_ADAM_BETA1} \
    --generator_adam_beta2=${GENERATOR_ADAM_BETA2} \
    --generator_adam_epsilon=${GENERATOR_ADAM_EPSILON} \
    --generator_clip_gradients=${GENERATOR_CLIP_GRADIENTS} \
    --generator_train_steps=${GENERATOR_TRAIN_STEPS} \
    \
    --use_minibatch_stddev=${USE_MINIBATCH_STDDEV} \
    --minibatch_stddev_group_size=${MINIBATCH_STDDEV_GROUP_SIZE} \
    --minibatch_stddev_averaging=${MINIBATCH_STDDEV_AVERAGING} \
    --discriminator_leaky_relu_alpha=${DISCRIMINATOR_LEAKY_RELU_ALPHA} \
    --discriminator_l1_regularization_scale=${DISCRIMINATOR_L1_REGULARIZATION_SCALE} \
    --discriminator_l2_regularization_scale=${DISCRIMINATOR_L2_REGULARIZATION_SCALE} \
    --discriminator_optimizer=${DISCRIMINATOR_OPTIMIZER} \
    --discriminator_learning_rate=${DISCRIMINATOR_LEARNING_RATE} \
    --discriminator_adam_beta1=${DISCRIMINATOR_ADAM_BETA1} \
    --discriminator_adam_beta2=${DISCRIMINATOR_ADAM_BETA2} \
    --discriminator_adam_epsilon=${DISCRIMINATOR_ADAM_EPSILON} \
    --discriminator_clip_gradients=${DISCRIMINATOR_CLIP_GRADIENTS} \
    --discriminator_gradient_penalty_coefficient=${DISCRIMINATOR_GRADIENT_PENALTY_COEFFICIENT} \
    --epsilon_drift=${EPSILON_DRIFT} \
    --discriminator_train_steps=${DISCRIMINATOR_TRAIN_STEPS} \
    \
    --encoder_leaky_relu_alpha=${ENCODER_LEAKY_RELU_ALPHA} \
    --encoder_l1_regularization_scale=${ENCODER_L1_REGULARIZATION_SCALE} \
    --encoder_l2_regularization_scale=${ENCODER_L2_REGULARIZATION_SCALE} \
    --encoder_optimizer=${ENCODER_OPTIMIZER} \
    --encoder_learning_rate=${ENCODER_LEARNING_RATE} \
    --encoder_adam_beta1=${ENCODER_ADAM_BETA1} \
    --encoder_adam_beta2=${ENCODER_ADAM_BETA2} \
    --encoder_adam_epsilon=${ENCODER_ADAM_EPSILON} \
    --encoder_clip_gradients=${ENCODER_CLIP_GRADIENTS}

In [1]:
import numpy as np
import tensorflow as tf

In [ ]:
# num_stages = 5
# num_steps_until_growth = 10
# with tf.Session() as sess:
#     # Create alpha variable to use for weighted sum for smooth fade-in.
#     with tf.variable_scope(name_or_scope="alpha", reuse=tf.AUTO_REUSE):
#         alpha_var = tf.get_variable(
#             name="alpha_var",
#             dtype=tf.float32,
#             # When the initializer is a function, tensorflow can place it
#             # "outside of the control flow context" to make sure it always runs.
#             initializer=lambda: tf.zeros(shape=[], dtype=tf.float32),
#             trainable=False
#         )

#     for growth_idx in range(num_stages):
#         for step in range(num_steps_until_growth):
#             global_step = growth_idx * num_steps_until_growth + step

#             if growth_idx > 0:
#                 if growth_idx % 2 == 1:
#                     # Update alpha var to linearly scale from 0 to 1 based on steps.
#                     alpha_var = tf.assign(
#                         ref=alpha_var,
#                         value=tf.divide(
#                             x=tf.cast(
#                                 # Add 1 since it trains on global step 0, so off by 1.
#                                 x=tf.add(
#                                     x=tf.mod(
#                                         x=global_step, y=num_steps_until_growth
#                                     ),
#                                     y=1
#                                 ),
#                                 dtype=tf.float32
#                             ),
#                             y=num_steps_until_growth
#                         ),
#                         name="alpha_var_update_op_assign_linear"
#                     )
#                 else:
#                     alpha_var = tf.assign(
#                         ref=alpha_var,
#                         value=tf.ones(shape=[], dtype=tf.float32),
#                         name="alpha_var_update_op_assign_ones"
#                     )
#             sess.run(tf.global_variables_initializer())
#             alpha_arr = sess.run(alpha_var)
#             print(global_step, alpha_arr)

In [9]:
conv_num_filters = 9
num_steps_until_growth = 10
train_steps = 200
num_stages = 2 * conv_num_filters - 1
min_train_steps_for_full_growth = num_stages * num_steps_until_growth
train_steps = max(train_steps, min_train_steps_for_full_growth)
with tf.Session() as sess:
    for global_step in range(train_steps):
        growth_index = tf.minimum(
            x=tf.cast(
                x=tf.floordiv(
                    x=global_step,
                    y=num_steps_until_growth,
                    name="global_step_floordiv"
                ),
                dtype=tf.int32),
            y=(conv_num_filters - 1) * 2,
            name="growth_index"
        )
        growth_index_arr = sess.run(growth_index)
        print(global_step, growth_index_arr, growth_index_arr % 2)

0 0 0
1 0 0
2 0 0
3 0 0
4 0 0
5 0 0
6 0 0
7 0 0
8 0 0
9 0 0
10 1 1
11 1 1
12 1 1
13 1 1
14 1 1
15 1 1
16 1 1
17 1 1
18 1 1
19 1 1
20 2 0
21 2 0
22 2 0
23 2 0
24 2 0
25 2 0
26 2 0
27 2 0
28 2 0
29 2 0
30 3 1
31 3 1
32 3 1
33 3 1
34 3 1
35 3 1
36 3 1
37 3 1
38 3 1
39 3 1
40 4 0
41 4 0
42 4 0
43 4 0
44 4 0
45 4 0
46 4 0
47 4 0
48 4 0
49 4 0
50 5 1
51 5 1
52 5 1
53 5 1
54 5 1
55 5 1
56 5 1
57 5 1
58 5 1
59 5 1
60 6 0
61 6 0
62 6 0
63 6 0
64 6 0
65 6 0
66 6 0
67 6 0
68 6 0
69 6 0
70 7 1
71 7 1
72 7 1
73 7 1
74 7 1
75 7 1
76 7 1
77 7 1
78 7 1
79 7 1
80 8 0
81 8 0
82 8 0
83 8 0
84 8 0
85 8 0
86 8 0
87 8 0
88 8 0
89 8 0
90 9 1
91 9 1
92 9 1
93 9 1
94 9 1
95 9 1
96 9 1
97 9 1
98 9 1
99 9 1
100 10 0
101 10 0
102 10 0
103 10 0
104 10 0
105 10 0
106 10 0
107 10 0
108 10 0
109 10 0
110 11 1
111 11 1
112 11 1
113 11 1
114 11 1
115 11 1
116 11 1
117 11 1
118 11 1
119 11 1
120 12 0
121 12 0
122 12 0
123 12 0
124 12 0
125 12 0
126 12 0
127 12 0
128 12 0
129 12 0
130 13 1
131 13 1
132 13 1
133 13 1
134 

## Predictions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
# Get data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print("x_train.shape = {}".format(x_train.shape))
print("y_train.shape = {}".format(y_train.shape))
print("x_test.shape = {}".format(x_test.shape))
print("y_test.shape = {}".format(y_test.shape))

In [ ]:
!gsutil ls gs://machine-learning-1234-bucket/gan/pg_anogan_sim_enc/trained_model/export/exporter | tail -1

In [ ]:
predict_fn = tf.contrib.predictor.from_saved_model(
    "gs://machine-learning-1234-bucket/gan/pg_anogan_sim_enc/trained_model/export/exporter/1591961908/"
)
predictions = predict_fn(
    {
        "query_image": x_test[0:10]
    }
)

In [ ]:
print(list(predictions.keys()))

In [ ]:
encoded_images = {k: v for k, v in predictions.items() if k[0:14] == "encoded_images"}
list(encoded_images.keys())

In [ ]:
anomaly_scores = {k: v for k, v in predictions.items() if k[0:14] == "anomaly_scores"}
list(anomaly_scores.keys())

In [ ]:
anomaly_flags = {k: v for k, v in predictions.items() if k[0:13] == "anomaly_flags"}
list(anomaly_flags.keys())

In [ ]:
def sort_dict_by_image_size(input_dict):
    """Sort predictions dictionary by image size.
    
    Args:
        input_dict: dict, contains prediction keys and array values.
    Return:
        Sorted input dictionary on image size in ascending order.
    """
    sorted_input_dict = [
        x[0:2]
        for x in sorted(
            [
                (
                    k,
                    input_dict[k],
                    int(k.split("x")[-1])
                )
                for k in input_dict.keys()
            ],
            key=lambda tup: tup[2]
        )
    ]

    for k, v in sorted_input_dict:
        print(k, v.shape)

    return sorted_input_dict

Convert image back to the original scale.

In [ ]:
encoded_images = {
    k: np.clip(
        a=((v + 1.0) * (255. / 2)).astype(np.int32),
        a_min=0,
        a_max=255
    )
    for k, v in encoded_images.items()
}

sorted_encoded_images = sort_dict_by_image_size(encoded_images)

In [ ]:
sorted_anomaly_scores = sort_dict_by_image_size(anomaly_scores)

In [ ]:
sorted_anomaly_flags = sort_dict_by_image_size(anomaly_flags)

In [ ]:
num_images = 5

for k, v in sorted_encoded_images:
    print(k)
    plt.figure(figsize=(int(k.split("x")[-1]), int(k.split("x")[-1])))
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(v[i], cmap=plt.cm.binary)
    plt.show()

In [ ]:
for k, v in sorted_anomaly_scores:
    print(k)
    print(v)

In [ ]:
for k, v in sorted_anomaly_flags:
    print(k)
    print(v)